In [1]:
import glob
import pandas as pd
import boto3
import re

In [2]:
path = 'scraper'
files = [file for file in glob.glob('../tours_data/*.csv')]
print(len(files))

23


In [3]:
li = []
for f in files:
    # read in csv
    temp_df = pd.read_csv(f)
    # append df to list
    li.append(temp_df)

In [4]:
data = pd.concat(li, axis=0)

In [5]:
data.head()

,Titles,Ratings,Number_Of_Ratings,Prices,Durations,Star_Ratings,Tour_Provider
0,Experience Real Bangkok by Bike,Excellent,1904 ratings,£22,3 hours,5.0,Viator
1,Private Guide In and Around Bangkok: Custom Tour,Excellent,1431 ratings,£78,8 hours,5.0,Viator
2,"Bangkok by Night: Temples, Markets and Food by...",Excellent,945 ratings,£54,4 hours,5.0,Viator
3,"Flexi Walking Temple Tour: Grand Palace, Wat P...",Excellent,794 ratings,£32,2 hours,5.0,GetYourGuide
4,"Bangkok by Night Tuk Tuk Tour: Markets, Temple...",Excellent,568 ratings,£57,4 hours,5.0,GetYourGuide


In [132]:
df = data.copy()

In [133]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4143 entries, 0 to 62
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Titles             4143 non-null   object 
 1   Ratings            4143 non-null   object 
 2   Number_Of_Ratings  4143 non-null   object 
 3   Prices             4143 non-null   object 
 4   Durations          4143 non-null   object 
 5   Star_Ratings       1197 non-null   float64
 6   Tour_Provider      4143 non-null   object 
dtypes: float64(1), object(6)
memory usage: 388.0+ KB


### Clean Data

In [134]:
# rename columns
df.columns = ['Title', 'Rating', 'Total_ratings',
              'Price(£)', 'Duration', 'Star_rating', 'Tour_provider']

In [135]:
df.head(2)

,Title,Rating,Total_ratings,Price(£),Duration,Star_rating,Tour_provider
0,Experience Real Bangkok by Bike,Excellent,1904 ratings,£22,3 hours,5.0,Viator
1,Private Guide In and Around Bangkok: Custom Tour,Excellent,1431 ratings,£78,8 hours,5.0,Viator


###### Clean Rating column

In [136]:
df.Rating.value_counts()

Excellent           2337
no data provided    1437
Very Good            270
Good                  59
Poor                  27
Fair                  13
Name: Rating, dtype: int64

In [137]:
df.Rating.replace('no data provided', 'NA', inplace=True)

In [138]:
df.Rating.value_counts()

Excellent    2337
NA           1437
Very Good     270
Good           59
Poor           27
Fair           13
Name: Rating, dtype: int64

###### Clean Total_ratings

In [ ]:
df['Total_ratings'].unique()

In [140]:
# clean number_of_ratings column to include only integers
df['Total_ratings'] = df['Total_ratings'].str.replace('ratings', '')

In [141]:
df.Total_ratings.value_counts()

no data provided    1705
1 rating             224
2                    119
3                    111
4                     81
                    ... 
1430                   1
1491                   1
1556                   1
1656                   1
61                     1
Name: Total_ratings, Length: 392, dtype: int64

In [142]:
df['Total_ratings'] = df['Total_ratings'].str.replace('rating', '')
df['Total_ratings'] = df['Total_ratings'].str.replace('no data provided', 'NA')

In [143]:
df.Total_ratings.value_counts()

NA       1705
1         224
2         119
3         111
4          81
         ... 
1430        1
1491        1
1556        1
1656        1
61          1
Name: Total_ratings, Length: 392, dtype: int64

###### Clean Prices

In [16]:
# remove £ symbol from prices
df['Price(£)'] = df['Price(£)'].str.replace('£', '')

In [17]:
df.head(2)

,Title,Rating,Total_ratings,Price(£),Duration,Star_rating,Tour_provider
0,Experience Real Bangkok by Bike,Excellent,1904,22,3 hours,5.0,Viator
1,Private Guide In and Around Bangkok: Custom Tour,Excellent,1431,78,8 hours,5.0,Viator


##### Clean Duration

In [ ]:
df.Duration.unique()

In [204]:
def get_duration_in_hours(duration):
    if 'hours' in duration:
        hours = duration.split(' ')[0]
        if '-' in hours:
            hour_interval = hours.split('-')
            lower_boundary = hour_interval[0]
            upper_boundary = hour_interval[-1]
            avg_hours = (int(lower_boundary) + int(upper_boundary))/2
            hours = int(round(avg_hours, 2))
        elif '_' not in hours:
            hours = int(hours)
        return hours
    elif 'hour' in duration:
        hour_split = duration.split(' ')[0]
        hour = int(hour_split)
        return hour
    elif 'minutes' in duration:
        minutes = duration.split(' ')[0]
        if '-' in minutes:
            minute_interval = minutes.split('-')
            lower_boundary = minute_interval[0]
            upper_boundary = minute_interval[-1]
            avg_minutes = (int(lower_boundary) + int(upper_boundary))/2
            minutes = round(int(avg_minutes)/60, 2)
        elif '_' not in minutes:
            minutes = round(int(minutes)/60, 2)
        return minutes 
    elif 'days' in duration:
        days = duration.split(' ')[0]
        if '-' in days:
            day_interval = days.split('-')
            lower_boundary = day_interval[0]
            upper_boundary = day_interval[-1]
            avg_days = (int(lower_boundary) + int(upper_boundary))/2
            days = round(int(avg_days) * 24, 2)  
        elif '_' not in days:
            days = int(days) * 24
        return days
    elif 'day' in duration:
        day_split = duration.split(' ')[0]
        day = int(day_split)
        return day
    else:
        return 'NA'
        

In [205]:
df['Duration(hours)'] = df['Duration'].map(get_duration_in_hours)

In [206]:
df.head()

,Title,Rating,Total_ratings,Price(£),Duration,Star_rating,Tour_provider,Duration(hours)
0,Experience Real Bangkok by Bike,Excellent,1904,£22,3 hours,5.0,Viator,3
1,Private Guide In and Around Bangkok: Custom Tour,Excellent,1431,£78,8 hours,5.0,Viator,8
2,"Bangkok by Night: Temples, Markets and Food by...",Excellent,945,£54,4 hours,5.0,Viator,4
3,"Flexi Walking Temple Tour: Grand Palace, Wat P...",Excellent,794,£32,2 hours,5.0,GetYourGuide,2
4,"Bangkok by Night Tuk Tuk Tour: Markets, Temple...",Excellent,568,£57,4 hours,5.0,GetYourGuide,4


In [208]:
df.drop(columns='Duration', inplace=True)